In [1]:
# Google Colab Setup
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input, LeakyReLU
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

Using TensorFlow backend.


In [0]:
os.chdir('/content/drive/My Drive/app/DCGAN-MNIST/')
os.environ["KERAS_BACKEND"] = 'tensorflow'
K.set_image_dim_ordering('th')

np.random.seed(1000)
randomDim = 100

In [4]:
#Load MNIST Data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train[:, np.newaxis, :, :]

11493376/11490434 [==============================] - 2s 0us/step


In [5]:
# Optimizer
adam = Adam(lr = 0.0002, beta_1 = 0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Generator Network
generator = Sequential()
generator.add(Dense(128 * 7 * 7, input_dim = randomDim, kernel_initializer = initializers.RandomNormal(stddev = 0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((128, 7, 7)))
generator.add(UpSampling2D(size = (2, 2)))
generator.add(Conv2D(64, kernel_size = (5, 5), padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size = (2, 2)))
generator.add(Conv2D(1, kernel_size = (5, 5), padding = 'same', activation = 'tanh'))
generator.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [7]:
# Discriminator Network
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size = (5, 5), strides = (2, 2), padding = 'same', input_shape = (1, 28, 28), kernel_initializer = initializers.RandomNormal(stddev = 0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size = (5, 5), strides = (2, 2), padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation = 'sigmoid'))
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# GAN network
discriminator.trainable = False
ganInput = Input(shape = (randomDim, ))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs = ganInput, outputs = ganOutput)
gan.compile(loss = 'binary_crossentropy', optimizer = adam)

In [0]:
dLosses = []
gLosses = []

In [0]:
# Plot the loss from each batch
def plotLoss(epoch):
  plt.figure(figsize = (10, 8))
  plt.plot(dLosses, label = 'Discriminative Loss')
  plt.plot(gLosses, label = 'Generative Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.savefig('dcgan_loss_epoch_%d.png' % epoch)
  plt.close()

In [0]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples = 100, dim = (10, 10), figsize = (10, 10)):
  noise = np.random.normal(0, 1, size = [examples, randomDim])
  generatedImages = generator.predict(noise)
  
  plt.figure(figsize = figsize)
  for i in range(generatedImages.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generatedImages[i, 0], interpolation = 'nearest', cmap = 'gray_r')
    plt.axis('off')
  
  plt.tight_layout()
  plt.savefig('dcgan_generated_image_epoch_%d.png' % epoch)
  plt.close()

In [0]:
def saveModels(epoch):
  generator.save('./Models/dcgan_generator_epoch_%d.h5' % epoch)
  discriminator.save('./Models/dcgan_discriminator_epoch_%d.h5' % epoch)

In [0]:
def train(epochs = 1, batchSize = 128):
  batchCount = X_train.shape[0] // batchSize
  print("Epochs : {}".format(epochs))
  print("Batch Size : {}".format(batchSize))
  print("Batches per epoch : {}".format(batchCount))
  
  for e in range(1, epochs + 1):
    stringDisplay = '-'*15 + 'Epoch ' + str(e) + '-'*15
    print(stringDisplay)
    
    for _ in tqdm(range(batchCount)):
      # Get a random set of input noise and images
      noise = np.random.normal(0, 1, size = [batchSize, randomDim])
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size = batchSize)]
      
      # Generate fake MNIST images
      generatedImages = generator.predict(noise)
      X = np.concatenate([imageBatch, generatedImages])
      
      # Labels for generated and real data
      yDis = np.zeros(2 * batchSize)
      yDis[:batchSize] = 0.9
      
      # Train discriminator
      discriminator.trainable = True
      dloss = discriminator.train_on_batch(X, yDis)
      
      # Train generator
      noise = np.random.normal(0, 1, size = [batchSize, randomDim])
      yGen = np.ones(batchSize)
      
      discriminator.trainable = False
      gloss = gan.train_on_batch(noise, yGen)
      
    # Store the loss of most recent batch from this epoch
    dLosses.append(dloss)
    gLosses.append(gloss)
  
    if e == 1 or e%5 == 0:
      plotGeneratedImages(e)
      saveModels(e)
  
  # Plot losses from every epoch
  plotLoss(e)
    
  

In [0]:
train(50, 128)